## Adopted from
### the notebook that <a href = "https://github.com/cjurban">Christopher J. Urban</a> kindly prepared. A big THANKS to him.

This notebook is for conducting amortized importance-weighted variational inference for exploratory item factor analysis, which is<br>
" ... computationally fast even in large data sets with many latent factors."<br>
"... The IWAE approximates the marginal maximum likelihood estimator using an importance sampling technique wherein increasing the number of importance-weighted (IW) samples drawn during fitting improves the approximation, typically at the cost of decreased computational efficiency."

Check out the publication by <a href = "https://scholar.google.com/citations?user=Q3glDmMAAAAJ&hl=en">Urban</a> & <a href = "https://dbauer.web.unc.edu/">Bauer</a>, which  is available 
<a href = "https://arxiv.org/pdf/2001.07859.pdf">here</a> and <a href = "https://link.springer.com/article/10.1007/s11336-021-09748-3">here</a>.<br>
The reproduction material for the paper is available @ <a href = "https://github.com/cjurban/DeepExploratoryIFA">Urban's Github repo</a>.

In [1]:
import pandas as pd

from __future__ import print_function
import torch
from torchvision import datasets, transforms
import sys
import timeit
from factor_analyzer import Rotator # May need to be installed; see https://pypi.org/project/factor-analyzer/
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
from pylab import *

from utils import *
from helper_layers import *
from base_class import *
from mirt_vae import *
from read_data import *

# Suppress scientific notation.
np.set_printoptions(suppress = True)

# Print full arrays.
np.set_printoptions(threshold = sys.maxsize)

# If CUDA is available, use it.
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")
kwargs = {"num_workers" : 1, "pin_memory" : True} if cuda else {}

# Set log interval.
log_interval = 200

In [2]:
csv_filename = "../000_data/df_efa_for_torch_nn_efa.csv"

# Full data set.
csv_loader = torch.utils.data.DataLoader(csv_dataset(csv_file = csv_filename, 
                                                     which_split = "full",
                                                     transform = to_tensor()),
                                         batch_size = 32, shuffle = True, **kwargs)

# Test data set.
csv_test_loader = torch.utils.data.DataLoader(csv_dataset(csv_file = csv_filename,
                                                          which_split = "test-only",
                                                          test_size = 0.025,
                                                          transform = to_tensor()),
                                              batch_size = 32, shuffle = True, **kwargs)

# Train data set.
csv_train_loader = torch.utils.data.DataLoader(csv_dataset(csv_file = csv_filename,
                                                           which_split = "train-only",
                                                           test_size = 0.025,
                                                           transform = to_tensor()),
                                               batch_size = 32, shuffle = True, **kwargs)

In [3]:
# Set random seeds.
seed = 666
torch.manual_seed(seed)
np.random.seed(seed)

# Initialize model.
print("\nStarting fitting")
start = timeit.default_timer()
dfm_vae = MIRTVAEClass(input_dim = 406, # number of columns in binarized data matrix
                       inference_model_dims = [136], # list of neural network hidden layer sizes
                       latent_dim = 3,
                       n_cats = [7] * 58, # list of number of categories for each item
                       learning_rate = 5e-3,
                       device = device,
                       log_interval = log_interval,
                       steps_anneal = 1e3)

"""
Fit model.
Note: iw_samples can be increased to improve the approximation to the MMLE.
As described in the paper, iw_samples = 5 seems to perform well in practice.
"""
dfm_vae.run_training(csv_loader, csv_test_loader, iw_samples = 5)
stop = timeit.default_timer()
run_time = stop - start
print("Fitting completed in", round(run_time, 2), "seconds")

# Extract estimated loadings and intercepts.
loadings = dfm_vae.model.loadings.weight.data.numpy()
intercepts = dfm_vae.model.intercepts.bias.data.numpy()

"""
Rotate loadings and extract factor correlations.
Note: In the paper I used geomin rotation, whereas here I use oblimin.
"""
rotator = Rotator(method = "oblimin")
rot_loadings = rotator.fit_transform(loadings)
cor_mat = rotator.phi_

"""
Compute approximate log-likelihood.
Note: This computation can be sped up or slowed down by decreasing or increasing
iw_samples, respectively. In the paper, I set iw_samples = 5000. 
"""
print("\nComputing approx. LL")
start = timeit.default_timer()
ll = dfm_vae.bic(csv_test_loader, iw_samples = 100)[1]
stop = timeit.default_timer()
print("Approx. LL computed in", round(stop - start, 2), "seconds")


Starting fitting
Fitting completed in 200.28 seconds

Computing approx. LL
Approx. LL computed in 1.38 seconds


In [4]:
norm_loadings = normalize_loadings(rot_loadings)
norm_loadings_df = pd.DataFrame(norm_loadings, columns = ['dim1', 'dim_2', 'dim_3'])
norm_loadings_df

,dim1,dim_2,dim_3
0,0.000501,0.005863,-0.001093
1,-0.001618,0.007902,0.000750
2,0.003509,-0.002859,0.001117
3,0.001359,0.001029,0.000415
4,0.001714,-0.005524,-0.000515
5,-0.000168,-0.001097,0.000157
6,0.001853,0.002267,0.000190
7,0.002374,-0.000769,0.001418
8,0.000929,0.000626,0.001186
9,0.001606,0.001438,-0.000993
